<h2> Quick walk through of tsfresh with financial data (Part I) </h2>

`tsfresh` is a powerful feature engineering package for time series data. But there are limited examples out there on how to use it on financial time series data. 

So here is a simple and easy to follow walk through on how to use `tsfresh` package on stock price data. 

<h3> functions in tsfresh package</h3>

I use `roll_time_series function` to prepare data and extract_features function to generate features from `Close` price data. I also use functions under `feature_calculators` module to validate generated features. 

* `roll_time_series` prepares the raw time series data in the format that can be processed by `extract_features` function.

* `extract_features` function requires the input dataset to have at least three columns: a `‘id’` column, a `‘time’` column, and a numeric column that new features will be generated from. Note that `‘id’` column and `‘time’` column do not need to be named as `‘id’` and `‘time’`.

* `feature_calculators` module includes all the functions encapsuled in `extract_features` function.  For example, `extract_features` returns a feature with suffix `‘abs_energy’`, the values in this feature are the same as the results of using feature_calculators.abs_energy function with same input data. 

***

<h4> 1. load data </h4>

load csv data into a pandas dataframe.

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [3]:
df = pd.read_csv('data/^GSPC.csv')

In [4]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2007-01-03,1418.030029,1429.420044,1407.859985,1416.599976,1416.599976,3429160000
1,2007-01-04,1416.599976,1421.839966,1408.430054,1418.339966,1418.339966,3004460000
2,2007-01-05,1418.339966,1418.339966,1405.750000,1409.709961,1409.709961,2919400000
3,2007-01-08,1409.260010,1414.979980,1403.969971,1412.839966,1412.839966,2763340000
4,2007-01-09,1412.839966,1415.609985,1405.420044,1412.109985,1412.109985,3038380000


In [5]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

<h4> 2. pre-process data </h4>

create dummy `id` column

In [6]:
#keep only Date and Close
df = df[['Date', 'Close']].copy(deep=True)

In [7]:
#create a dummy id column
df['id']=1

In [8]:
df.head()

,Date,Close,id
0,2007-01-03,1416.599976,1
1,2007-01-04,1418.339966,1
2,2007-01-05,1409.709961,1
3,2007-01-08,1412.839966,1
4,2007-01-09,1412.109985,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3301 entries, 0 to 3300
Data columns (total 3 columns):
Date     3301 non-null object
Close    3301 non-null float64
id       3301 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 77.5+ KB


<h4> 3. prepare data using roll_time_series </h4>


1. `id` column will be converted into a tuple; 
1. `Date` columns is used in `column_sort` parameter.
1. `max_timeshift` and `min_timeshift`: must be a integer `>=` than `min_timeshift`
 - setup 1: `max_timeshift=6, min_timeshift=6`
 - setup 2: `max_timeshift=6, min_timeshift=4`


In [10]:
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh import extract_features

In [11]:
df_rolled = roll_time_series(df, column_id="id", column_sort="Date", max_timeshift=6, min_timeshift=6, disable_progressbar=True)

In [12]:
df_rolled.head(7)

,Date,Close,id
0,2007-01-03,1416.599976,"(1, 2007-01-11)"
1,2007-01-04,1418.339966,"(1, 2007-01-11)"
2,2007-01-05,1409.709961,"(1, 2007-01-11)"
3,2007-01-08,1412.839966,"(1, 2007-01-11)"
4,2007-01-09,1412.109985,"(1, 2007-01-11)"
5,2007-01-10,1414.849976,"(1, 2007-01-11)"
6,2007-01-11,1423.819946,"(1, 2007-01-11)"


In [13]:
print(df.shape, df_rolled.shape)

(3301, 3) (23065, 3)


In [14]:
df.head()

,Date,Close,id
0,2007-01-03,1416.599976,1
1,2007-01-04,1418.339966,1
2,2007-01-05,1409.709961,1
3,2007-01-08,1412.839966,1
4,2007-01-09,1412.109985,1


In [15]:
df_rolled2 = roll_time_series(df, column_id="id", column_sort="Date", max_timeshift=6, min_timeshift=4, disable_progressbar=True)

In [16]:
df_rolled2.head(7)

,Date,Close,id
0,2007-01-03,1416.599976,"(1, 2007-01-09)"
1,2007-01-04,1418.339966,"(1, 2007-01-09)"
2,2007-01-05,1409.709961,"(1, 2007-01-09)"
3,2007-01-08,1412.839966,"(1, 2007-01-09)"
4,2007-01-09,1412.109985,"(1, 2007-01-09)"
5,2007-01-03,1416.599976,"(1, 2007-01-10)"
6,2007-01-04,1418.339966,"(1, 2007-01-10)"


<h5> compare the two setups of max_timeshift and min_timeshift </h5>

1. setup 1: `max_timeshift=6, min_timeshift=6`
1. setup 2: `max_timeshift=6, min_timeshift=4`

In [17]:
df_rolled[df_rolled['Date']=='2007-01-03']

,Date,Close,id
0,2007-01-03,1416.599976,"(1, 2007-01-11)"


In [18]:
df_rolled2[df_rolled2['Date']=='2007-01-03']

,Date,Close,id
0,2007-01-03,1416.599976,"(1, 2007-01-09)"
5,2007-01-03,1416.599976,"(1, 2007-01-10)"
11,2007-01-03,1416.599976,"(1, 2007-01-11)"


In [19]:
df_rolled.tail(3)

,Date,Close,id
23062,2020-02-10,3352.090088,"(1, 2020-02-12)"
23063,2020-02-11,3357.750000,"(1, 2020-02-12)"
23064,2020-02-12,3379.449951,"(1, 2020-02-12)"


In [20]:
df_rolled2.tail(3)

,Date,Close,id
23073,2020-02-10,3352.090088,"(1, 2020-02-12)"
23074,2020-02-11,3357.750000,"(1, 2020-02-12)"
23075,2020-02-12,3379.449951,"(1, 2020-02-12)"


In [22]:
print(df.shape, df_rolled.shape, df_rolled2.shape)

(3301, 3) (23065, 3) (23076, 3)


In [23]:
df_rolled[df_rolled['id']==df_rolled['id'].values[0]]

,Date,Close,id
0,2007-01-03,1416.599976,"(1, 2007-01-11)"
1,2007-01-04,1418.339966,"(1, 2007-01-11)"
2,2007-01-05,1409.709961,"(1, 2007-01-11)"
3,2007-01-08,1412.839966,"(1, 2007-01-11)"
4,2007-01-09,1412.109985,"(1, 2007-01-11)"
5,2007-01-10,1414.849976,"(1, 2007-01-11)"
6,2007-01-11,1423.819946,"(1, 2007-01-11)"


In [24]:
df_rolled2[df_rolled2['id']==df_rolled['id'].values[0]]

,Date,Close,id
11,2007-01-03,1416.599976,"(1, 2007-01-11)"
12,2007-01-04,1418.339966,"(1, 2007-01-11)"
13,2007-01-05,1409.709961,"(1, 2007-01-11)"
14,2007-01-08,1412.839966,"(1, 2007-01-11)"
15,2007-01-09,1412.109985,"(1, 2007-01-11)"
16,2007-01-10,1414.849976,"(1, 2007-01-11)"
17,2007-01-11,1423.819946,"(1, 2007-01-11)"


In [25]:
df_rolled[df_rolled['Date']=='2007-01-10']

,Date,Close,id
5,2007-01-10,1414.849976,"(1, 2007-01-11)"
11,2007-01-10,1414.849976,"(1, 2007-01-12)"
17,2007-01-10,1414.849976,"(1, 2007-01-16)"
23,2007-01-10,1414.849976,"(1, 2007-01-17)"
29,2007-01-10,1414.849976,"(1, 2007-01-18)"
35,2007-01-10,1414.849976,"(1, 2007-01-19)"


In [26]:
df_rolled2[df_rolled2['Date']=='2007-01-10']

,Date,Close,id
10,2007-01-10,1414.849976,"(1, 2007-01-10)"
16,2007-01-10,1414.849976,"(1, 2007-01-11)"
22,2007-01-10,1414.849976,"(1, 2007-01-12)"
28,2007-01-10,1414.849976,"(1, 2007-01-16)"
34,2007-01-10,1414.849976,"(1, 2007-01-17)"
40,2007-01-10,1414.849976,"(1, 2007-01-18)"
46,2007-01-10,1414.849976,"(1, 2007-01-19)"


<h4> 4. create new features using extract_features </h4>

In [27]:
df_features = extract_features(df_rolled, column_id="id", column_sort="Date")

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:40<00:00,  2.05s/it]


In [28]:
df_features2 = extract_features(df_rolled2, column_id="id", column_sort="Date")

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:39<00:00,  1.96s/it]


In [29]:
print(df_features.shape, df_features2.shape)

(3295, 779) (3297, 779)


In [30]:
df_features.head()

Close__variance_larger_than_standard_deviation  \
1 2007-01-11                                             1.0   
  2007-01-12                                             1.0   
  2007-01-16                                             1.0   
  2007-01-17                                             1.0   
  2007-01-18                                             1.0   

              Close__has_duplicate_max  Close__has_duplicate_min  \
1 2007-01-11                       0.0                       0.0   
  2007-01-12                       0.0                       0.0   
  2007-01-16                       0.0                       0.0   
  2007-01-17                       0.0                       0.0   
  2007-01-18                       0.0                       0.0   

              Close__has_duplicate  Close__sum_values  Close__abs_energy  \
1 2007-01-11                   0.0        9908.269776       1.402496e+07   
  2007-01-12                   0.0        9922.399780       1.406519e+07   
  2007-01-16                   0.0        9935.959838       1.410384e+07   
  2007-01-17                   0.0        9956.869872       1.416323e+07   
  2007-01-18                   0.0        9970.399901       1.420165e+07   

              Close__mean_abs_change  Close__mean_change  \
1 2007-01-11                4.323324            1.203328   
  2007-01-12                5.184998            2.065002   
  2007-01-16                3.941671            3.698344   
  2007-01-17                3.633341            2.963338   
  2007-01-18                4.220011            2.376668   

              Close__mean_second_derivative_central  Close__median  \
1 2007-01-11                               0.722998    1414.849976   
  2007-01-12                               1.554004    1414.849976   
  2007-01-16                              -0.195996    1414.849976   
  2007-01-17                              -0.055005    1423.819946   
  2007-01-18                              -0.698999    1426.369995   

              Close__mean  Close__length  Close__standard_deviation  \
1 2007-01-11  1415.467111            7.0                   4.326073   
  2007-01-12  1417.485683            7.0                   6.909142   
  2007-01-16  1419.422834            7.0                   8.576790   
  2007-01-17  1422.409982            7.0                   8.310960   
  2007-01-18  1424.342843            7.0                   7.381920   

              Close__variation_coefficient  Close__variance  Close__skewness  \
1 2007-01-11                      0.003056        18.714907         0.825436   
  2007-01-12                      0.004874        47.736244         1.029587   
  2007-01-16                      0.006042        73.561323         0.523817   
  2007-01-17                      0.005843        69.072059        -0.133691   
  2007-01-18                      0.005183        54.492749        -0.800161   

              Close__kurtosis  Close__absolute_sum_of_changes  \
1 2007-01-11         0.679915                       25.939942   
  2007-01-12         0.224793                       31.109986   
  2007-01-16        -1.913231                       23.650025   
  2007-01-17        -2.499627                       21.800049   
  2007-01-18        -1.126892                       25.320068   

              Close__longest_strike_below_mean  \
1 2007-01-11                               4.0   
  2007-01-12                               4.0   
  2007-01-16                               4.0   
  2007-01-17                               3.0   
  2007-01-18                               3.0   

              Close__longest_strike_above_mean  Close__count_above_mean  \
1 2007-01-11                               2.0                      3.0   
  2007-01-12                               2.0                      3.0   
  2007-01-16                               3.0                      3.0   
  2007-01-17                               4.0                      4.0   
  2007

In [31]:
df_features2.head()

Close__variance_larger_than_standard_deviation  \
1 2007-01-09                                             1.0   
  2007-01-10                                             1.0   
  2007-01-11                                             1.0   
  2007-01-12                                             1.0   
  2007-01-16                                             1.0   

              Close__has_duplicate_max  Close__has_duplicate_min  \
1 2007-01-09                       0.0                       0.0   
  2007-01-10                       0.0                       0.0   
  2007-01-11                       0.0                       0.0   
  2007-01-12                       0.0                       0.0   
  2007-01-16                       0.0                       0.0   

              Close__has_duplicate  Close__sum_values  Close__abs_energy  \
1 2007-01-09                   0.0        7069.599854       9.995897e+06   
  2007-01-10                   0.0        8484.449830       1.199770e+07   
  2007-01-11                   0.0        9908.269776       1.402496e+07   
  2007-01-12                   0.0        9922.399780       1.406519e+07   
  2007-01-16                   0.0        9935.959838       1.410384e+07   

              Close__mean_abs_change  Close__mean_change  \
1 2007-01-09                3.557495           -1.122498   
  2007-01-10                3.393994           -0.350000   
  2007-01-11                4.323324            1.203328   
  2007-01-12                5.184998            2.065002   
  2007-01-16                3.941671            3.698344   

              Close__mean_second_derivative_central  Close__median  \
1 2007-01-09                              -0.411662    1412.839966   
  2007-01-10                               0.125000    1413.844971   
  2007-01-11                               0.722998    1414.849976   
  2007-01-12                               1.554004    1414.849976   
  2007-01-16                              -0.195996    1414.849976   

              Close__mean  Close__length  Close__standard_deviation  \
1 2007-01-09  1413.919971            5.0                   3.126833   
  2007-01-10  1414.074972            6.0                   2.875360   
  2007-01-11  1415.467111            7.0                   4.326073   
  2007-01-12  1417.485683            7.0                   6.909142   
  2007-01-16  1419.422834            7.0                   8.576790   

              Close__variation_coefficient  Close__variance  Close__skewness  \
1 2007-01-09                      0.002211         9.777085         0.232021   
  2007-01-10                      0.002033         8.267697         0.014460   
  2007-01-11                      0.003056        18.714907         0.825436   
  2007-01-12                      0.004874        47.736244         1.029587   
  2007-01-16                      0.006042        73.561323         0.523817   

              Close__kurtosis  Close__absolute_sum_of_changes  \
1 2007-01-09        -1.643811                       14.229981   
  2007-01-10        -0.900978                       16.969972   
  2007-01-11         0.679915                       25.939942   
  2007-01-12         0.224793                       31.109986   
  2007-01-16        -1.913231                       23.650025   

              Close__longest_strike_below_mean  \
1 2007-01-09                               3.0   
  2007-01-10                               3.0   
  2007-01-11                               4.0   
  2007-01-12                               4.0   
  2007-01-16                               4.0   

              Close__longest_strike_above_mean  Close__count_above_mean  \
1 2007-01-09                               2.0                      2.0   
  2007-01-10                               2.0                      3.0   
  2007-01-11                               2.0                      3.0   
  2007-01-12                               2.0                      3.0   
  2007

In [32]:
df_features.tail(5)

Close__variance_larger_than_standard_deviation  \
1 2020-02-06                                             1.0   
  2020-02-07                                             1.0   
  2020-02-10                                             1.0   
  2020-02-11                                             1.0   
  2020-02-12                                             1.0   

              Close__has_duplicate_max  Close__has_duplicate_min  \
1 2020-02-06                       0.0                       0.0   
  2020-02-07                       0.0                       0.0   
  2020-02-10                       0.0                       0.0   
  2020-02-11                       0.0                       0.0   
  2020-02-12                       0.0                       0.0   

              Close__has_duplicate  Close__sum_values  Close__abs_energy  \
1 2020-02-06                   0.0       23009.559814       7.564553e+07   
  2020-02-07                   0.0       23063.869873       7.600404e+07   
  2020-02-10                   0.0       23132.300049       7.645812e+07   
  2020-02-11                   0.0       23264.530029       7.732863e+07   
  2020-02-12                   0.0       23395.060058       7.819383e+07   

              Close__mean_abs_change  Close__mean_change  \
1 2020-02-06               31.443319           12.063354   
  2020-02-07               32.744995            7.341675   
  2020-02-10               27.118367           21.095011   
  2020-02-11               24.161702           18.138346   
  2020-02-12               19.666666           13.643310   

              Close__mean_second_derivative_central  Close__median  \
1 2020-02-06                               0.083008    3283.659912   
  2020-02-07                               4.006982    3297.590088   
  2020-02-10                               0.098022    3327.709961   
  2020-02-11                              -4.301025    3334.689941   
  2020-02-12                              -1.539990    3345.780029   

              Close__mean  Close__length  Close__standard_deviation  \
1 2020-02-06  3287.079973            7.0                  40.120402   
  2020-02-07  3294.838553            7.0                  41.934999   
  2020-02-10  3304.614293            7.0                  45.971466   
  2020-02-11  3323.504290            7.0                  35.583514   
  2020-02-12  3342.151437            7.0                  23.893758   

              Close__variation_coefficient  Close__variance  Close__skewness  \
1 2020-02-06                      0.012205      1609.646659         0.057011   
  2020-02-07                      0.012727      1758.544174        -0.502863   
  2020-02-10                      0.013911      2113.375693        -0.860719   
  2020-02-11                      0.010707      1266.186474        -1.464097   
  2020-02-12                      0.007149       570.911679        -0.477794   

              Close__kurtosis  Close__absolute_sum_of_changes  \
1 2020-02-06        -0.961725                      188.659911   
  2020-02-07        -1.189890                      196.469969   
  2020-02-10        -0.961861                      162.710204   
  2020-02-11         1.807174                      144.970214   
  2020-02-12         0.905277                      117.999999   

              Close__longest_strike_below_mean  \
1 2020-02-06                               4.0   
  2020-02-07                               3.0   
  2020-02-10                               3.0   
  2020-02-11                               2.0   
  2020-02-12                               2.0   

              Close__longest_strike_above_mean  Close__count_above_mean  \
1 2020-02-06                               3.0                      3.0   
  2020-02-07                               4.0                      4.0   
  2020-02-10                               4.0                      4.0   
  2020-02-11                               5.0                      5.0   
  2020

In [33]:
df_features2.tail(5)

Close__variance_larger_than_standard_deviation  \
1 2020-02-06                                             1.0   
  2020-02-07                                             1.0   
  2020-02-10                                             1.0   
  2020-02-11                                             1.0   
  2020-02-12                                             1.0   

              Close__has_duplicate_max  Close__has_duplicate_min  \
1 2020-02-06                       0.0                       0.0   
  2020-02-07                       0.0                       0.0   
  2020-02-10                       0.0                       0.0   
  2020-02-11                       0.0                       0.0   
  2020-02-12                       0.0                       0.0   

              Close__has_duplicate  Close__sum_values  Close__abs_energy  \
1 2020-02-06                   0.0       23009.559814       7.564553e+07   
  2020-02-07                   0.0       23063.869873       7.600404e+07   
  2020-02-10                   0.0       23132.300049       7.645812e+07   
  2020-02-11                   0.0       23264.530029       7.732863e+07   
  2020-02-12                   0.0       23395.060058       7.819383e+07   

              Close__mean_abs_change  Close__mean_change  \
1 2020-02-06               31.443319           12.063354   
  2020-02-07               32.744995            7.341675   
  2020-02-10               27.118367           21.095011   
  2020-02-11               24.161702           18.138346   
  2020-02-12               19.666666           13.643310   

              Close__mean_second_derivative_central  Close__median  \
1 2020-02-06                               0.083008    3283.659912   
  2020-02-07                               4.006982    3297.590088   
  2020-02-10                               0.098022    3327.709961   
  2020-02-11                              -4.301025    3334.689941   
  2020-02-12                              -1.539990    3345.780029   

              Close__mean  Close__length  Close__standard_deviation  \
1 2020-02-06  3287.079973            7.0                  40.120402   
  2020-02-07  3294.838553            7.0                  41.934999   
  2020-02-10  3304.614293            7.0                  45.971466   
  2020-02-11  3323.504290            7.0                  35.583514   
  2020-02-12  3342.151437            7.0                  23.893758   

              Close__variation_coefficient  Close__variance  Close__skewness  \
1 2020-02-06                      0.012205      1609.646659         0.057011   
  2020-02-07                      0.012727      1758.544174        -0.502863   
  2020-02-10                      0.013911      2113.375693        -0.860719   
  2020-02-11                      0.010707      1266.186474        -1.464097   
  2020-02-12                      0.007149       570.911679        -0.477794   

              Close__kurtosis  Close__absolute_sum_of_changes  \
1 2020-02-06        -0.961725                      188.659911   
  2020-02-07        -1.189890                      196.469969   
  2020-02-10        -0.961861                      162.710204   
  2020-02-11         1.807174                      144.970214   
  2020-02-12         0.905277                      117.999999   

              Close__longest_strike_below_mean  \
1 2020-02-06                               4.0   
  2020-02-07                               3.0   
  2020-02-10                               3.0   
  2020-02-11                               2.0   
  2020-02-12                               2.0   

              Close__longest_strike_above_mean  Close__count_above_mean  \
1 2020-02-06                               3.0                      3.0   
  2020-02-07                               4.0                      4.0   
  2020-02-10                               4.0                      4.0   
  2020-02-11                               5.0                      5.0   
  2020

<h4> 5. validate new features using feature_calculators </h4>

In [34]:
from tsfresh.feature_extraction import feature_calculators

In [36]:
#compare with Close__sum_values in setup 1 and 2
df.iloc[-7:, ].Close.sum(), df.iloc[-7:, ].Close.sum()

(23395.060058, 23395.060058)

In [37]:
#compare mean_abs_change 
val1 = feature_calculators.mean_abs_change(df.iloc[-7:, ].Close)
val2 = feature_calculators.mean_abs_change(df.iloc[-7:, ].Close)
print(val1, val2)

19.666666499999945 19.666666499999945


In [38]:
#compare mean_change
val1 = feature_calculators.mean_change(df.iloc[-7:, ].Close)
val2 = feature_calculators.mean_change(df.iloc[-7:, ].Close)
print(val1, val2)

13.643310499999947 13.643310499999947


<h4> 6. conclusion </h4>

`max_timeshift` and `min_timeshift`:
 - the two setups are essentially the same except that the second setup has two more rows after applying `extract_features` function

In [39]:
df_features.to_csv('data/new_features.csv', sep='|')